# Vertex AI - External Model Serving

This notebook uses a Vertex AI Notebook to train a TensorFlow model (locally) with the data in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then saved and Vertex AI clients are used to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-04.png">

---
## Setup

Prepare TensorFlow:

In [1]:
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf

Setup Parameters

In [2]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

MODEL_DIR='gs://{}/digits/model/04_keras'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

BATCH_SIZE = 30

MODEL_NAME='MODEL_04_KERAS-DIGITS'
ENDPOINT_NAME='ENDPOINT_04_KERAS-DIGITS'
params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup Vertex AI Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [3]:
from google.cloud import aiplatform

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Prepare Data Connection

Retrieve the Schema info from BigQuery Information Schema via the Storage API:
- https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas

In [4]:
from google.cloud import bigquery
bqclient = bigquery.Client()
bqjob = bqclient.query(
"""
SELECT * FROM `"""+BQDATASET_ID+""".INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE TABLE_NAME = '"""+BQTABLE_ID+"""' """
)
schema = bqjob.result().to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_prepped,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_prepped,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_prepped,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_prepped,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_prepped,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_prepped,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_prepped,target,target,INT64,None
65,statmike-mlops,digits,digits_prepped,target_OE,target_OE,STRING,None


Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [5]:
OMIT = ['target_OE','SPLITS']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into features, target and one_hot encodes the `target`:

In [6]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [7]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(BATCH_SIZE)

In [8]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(BATCH_SIZE)

Review a single batch of the train data:

In [9]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns: ',columns)
    print('target: ',b)

columns:  ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']
target:  tf.Tensor(
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0

---
## Train the Model

Define the Model:

In [10]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

In [11]:
#model.summary()

Fit the Model:

In [12]:
history = model.fit(train,epochs=25)

Epoch 1/25
47/47 [==============================] - 2s 5ms/step - loss: 3.7176 - accuracy: 0.4720
Epoch 2/25
47/47 [==============================] - 0s 4ms/step - loss: 0.7248 - accuracy: 0.8077
Epoch 3/25
47/47 [==============================] - 0s 4ms/step - loss: 0.4121 - accuracy: 0.8708
Epoch 4/25
47/47 [==============================] - 0s 4ms/step - loss: 0.3156 - accuracy: 0.9077
Epoch 5/25
47/47 [==============================] - 0s 4ms/step - loss: 0.2465 - accuracy: 0.9297
Epoch 6/25
47/47 [==============================] - 0s 3ms/step - loss: 0.2132 - accuracy: 0.9368
Epoch 7/25
47/47 [==============================] - 0s 4ms/step - loss: 0.1788 - accuracy: 0.9475
Epoch 8/25
47/47 [==============================] - 0s 4ms/step - loss: 0.1742 - accuracy: 0.9461
Epoch 9/25
47/47 [==============================] - 0s 4ms/step - loss: 0.1415 - accuracy: 0.9560
Epoch 10/25
47/47 [==============================] - 0s 4ms/step - loss: 0.1267 - accuracy: 0.9617
Epoch 11/25
47/47 [

Evaluate the model with the test data:

In [13]:
loss, accuracy = model.evaluate(test)

13/13 [==============================] - 1s 5ms/step - loss: 0.2723 - accuracy: 0.9201


Create Prediction from a batch of the test data:

In [14]:
model.predict(test.take(1))

array([[3.90482114e-07, 7.16976821e-01, 8.91303871e-12, 6.33650288e-08,
        2.37087036e-07, 1.21065058e-09, 3.15081405e-09, 2.45950985e-11,
        8.11138889e-05, 2.82941371e-01],
       [7.30918472e-13, 8.98274064e-01, 2.07599804e-09, 2.86356749e-06,
        1.09785587e-05, 4.70925966e-15, 4.98486006e-05, 5.28450066e-08,
        1.01661623e-01, 5.29738543e-07],
       [4.73527422e-07, 9.82992470e-01, 7.56323684e-11, 4.10787315e-08,
        3.85962807e-07, 1.33955708e-10, 4.35669445e-10, 2.88856050e-10,
        4.87712023e-05, 1.69579294e-02],
       [3.48886441e-11, 2.59025306e-01, 6.00855765e-06, 1.67152160e-04,
        7.57343832e-06, 1.98942459e-14, 4.30315384e-04, 2.30328351e-06,
        7.40355194e-01, 6.17956266e-06],
       [2.39083042e-09, 9.78541672e-01, 1.09199295e-02, 7.20863085e-08,
        3.84722898e-06, 1.00638009e-18, 9.01380076e-07, 6.01969799e-03,
        4.51388024e-03, 5.41500000e-08],
       [1.33139324e-11, 2.47675180e-05, 3.11547781e-15, 7.82471532e-10,
   

---
## Save the model:

In [15]:
model.save(MODEL_DIR)

INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/model/04_keras/assets
INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/model/04_keras/assets


---
## Upload the Model to Vertex AI

Create a client to the Model Service, define the Model, and upload the model:

In [16]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

MODEL = {
    "display_name": MODEL_NAME,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Retrieve the model information and view the name and display name:

In [17]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.display_name, model_info.name

('MODEL_04_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/models/2450829010498748416')

---
## Create the Vertex AI Endpoint

Create a client to the Endpoint Service and use it to create the endpoint:

In [18]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

Retrieve the endpoint information and view the name and display name:

In [19]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.display_name, endpoint_info.name

('ENDPOINT_04_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/endpoints/2069527174079053824')

---
## Deploy the Model to the Vertex AI Endpoint

In [20]:
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+MODEL_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 1,
            "machine_spec": {
                    "machine_type": DEPLOY_COMPUTE,
                    "accelerator_count": 0,
                }
        }   
}

TRAFFIC = {
    '0' : 100
}

dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

In [33]:
dmodel_info = dmodel.result().deployed_model
dmodel_info.id

'9005800675950460928'

Retrieve the deployed model information from the endpoint:

In [22]:
clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models

[id: "9005800675950460928"
model: "projects/691911073727/locations/us-central1/models/2450829010498748416"
display_name: "DEPLOYED_MODEL_04_KERAS-DIGITS"
create_time {
  seconds: 1628767645
  nanos: 734395000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

---
## Predictions

Create a client to the prediction service:

In [23]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [24]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

Downloading: 100%|██████████| 388/388 [00:00<00:00, 389.98rows/s]


In [25]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,0.0,7.0,16.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,16.0,6.0,0.0,1,Odd,TEST


In [26]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

### With Python Client

Request prediction from the prediction service:

In [27]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [28]:
response.predictions

[[3.90481176e-07, 0.716977239, 8.91301095e-12, 6.33650643e-08, 2.37086724e-07, 1.21064903e-09, 3.15080984e-09, 2.45950638e-11, 8.11139398e-05, 0.282940984]]

In [29]:
import numpy as np
np.argmax(response.predictions[0])

1

### With REST

In [30]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [31]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{API_ENDPOINT}/v1/{endpoint_info.name}:predict

{
  "predictions": [
    [
      3.90481176e-07,
      0.716977239,
      8.91301095e-12,
      6.33650643e-08,
      2.37086724e-07,
      1.21064903e-09,
      3.15080984e-09,
      2.45950638e-11,
      8.11139398e-05,
      0.282940984
    ]
  ],
  "deployedModelId": "9005800675950460928"
}


### With gcloud

In [32]:
!gcloud beta ai endpoints predict {endpoint_info.name.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[3.90481176e-07, 0.716977239, 8.91301095e-12, 6.33650643e-08, 2.37086724e-07, 1.21064903e-09, 3.15080984e-09, 2.45950638e-11, 8.11139398e-05, 0.282940984]]


---
## Remove Resources
see notebook "XX - Cleanup"